In [4]:
import os
os.chdir('E:\学习\python\py_codbase\PK_LLM')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

import pandas as pd
from PK_LLM_endfront.evaluation.eva_funcs import Evaluator,Record,DataLoader
from PK_LLM_endfront.evaluation.llms import PredictorFactory
from ragas.metrics import answer_correctness

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
prompt = PromptTemplate(
    input_variables=['ans','re_ans'],
    template="""
    
Original Answer:
{ans}

Reference Material:
{re_ans}

Steps to Improve the Original Answer Using Reference Material:

Add Details: Supplement the answer with details from the reference material to make it more comprehensive and in-depth.
Adjust Expression: Enhance the clarity and logic of the language used in the answer, avoid modal words like "might" or "perhaps" to ensure assertiveness.
Highlight Key Points: Emphasize the most important points or information in the answer to ensure they are adequately presented.
Check Coherence: Ensure that the information in the answer is logically coherent and flows naturally.
Output Requirements:

Answer Content Only: Ensure the output includes only the answer.
No Modal Words: Avoid using words that could weaken the certainty of the answer.

"""
)
chain = prompt | ChatOpenAI(temperature=0) | StrOutputParser()

In [8]:
import pandas as pd
import json
from tqdm import tqdm
for i in range(1,7):
    if i <3:
        continue
    data = DataLoader.load_json(f"_data\\testqa\\testdata_{1}.json")
    df = pd.read_csv(f"eval_reasult2\\{i}\\ablat1_eval_rag_context.csv")
    output_file = f"_data\\testqa2\\testdata_{i}.json"
    lis = []
    for _i,qa in tqdm(enumerate(data)):
        answer = qa['Answer']
        _answer = chain.invoke({'ans':answer,'re_ans':df['prediction'].loc[_i]})
        qa['Answer'] = _answer
        lis.append(qa)
    with open(output_file, 'w', encoding='utf-8') as file:
                    json.dump(lis, file, indent=4)


0it [00:00, ?it/s]

50it [04:44,  5.69s/it]
50it [04:30,  5.40s/it]
50it [04:11,  5.04s/it]
50it [03:58,  4.78s/it]


In [5]:
d = DataLoader(f"_data\\testqa\\testdata_{1}.json")
r = next(d.get_data()) # record demo
e = Evaluator()
print(r.data['Question'])
e.record_lis = []
p1 = PredictorFactory().get_predictor(model_type='gpt-3.5-turbo')
r.add_prediction(p1)
r.data['Answer'] = chain.invoke({'ans':r.data['Answer'],'re_ans':df['prediction'].loc[0]})
e.add_Record(r)
e.evaluate()
print(r.evaluation['answer_correctness'])
print(r.evaluation['answer_similarity'])




What are the benefits and challenges of using synergistic sample preparation and multiple analytical techniques in untargeted metabolomics?


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

0.675747415221084
0.9529896608843359


In [4]:
d = DataLoader(f"_data\\testqa\\testdata_{1}.json")
r = next(d.get_data()) # record demo
import pandas as pd
df = pd.read_csv("eval_reasult\\1\\new_eval_rag.csv")
e = Evaluator()
print(r.data['Question'])
e.record_lis = []
r.prediction = df['prediction'].loc[0]
r.data['Answer'] = chain.invoke({'ans':r.data['Answer'],'re_ans':df['prediction'].loc[0]})
e.add_Record(r)
e.evaluate()
print(r.evaluation['answer_correctness'])
print(r.evaluation['answer_similarity'])

What are the benefits and challenges of using synergistic sample preparation and multiple analytical techniques in untargeted metabolomics?


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

0.7431619883252342
0.9726479533009366


In [11]:
r.data['Answer']

'Using synergistic sample preparation and multiple analytical techniques in untargeted metabolomics offers several benefits. Firstly, it can significantly reduce sample preparation time, with some methods bringing it down to as low as 30 minutes. The incorporation of multidimensional chromatography with At-column dilution can also streamline the process from sample acquisition to analysis, enhancing efficiency. Moreover, this approach enables the sensitive detection of analyte concentrations at sub ng/mL levels, leading to more precise and comprehensive results.\n\nHowever, the text did not explicitly mention any challenges associated with using synergistic sample preparation and multiple analytical techniques in untargeted metabolomics. This indicates that while there are numerous benefits to this approach, potential challenges were not highlighted in the provided context.\n\nOverall, the combination of synergistic sample preparation and multiple analytical techniques in untargeted me

In [12]:
r.prediction

'The benefits of using synergistic sample preparation and multiple analytical techniques in untargeted metabolomics include improved number of metabolites reliably detected and correctly assigned, better annotation decisions, and addressing the issues of reproducibility found in single platform methods. Additionally, this approach can significantly reduce sample preparation time, with some methods bringing it down to as low as 30 minutes. The incorporation of multidimensional chromatography with At-column dilution can also streamline the process from sample acquisition to analysis, enhancing efficiency. Moreover, this method enables the sensitive detection of analyte concentrations at sub ng/mL levels, leading to more precise and comprehensive results.\n\nHowever, challenges associated with this approach include the analysis of large data sets from disparate analytical techniques, which presents unique challenges. While the general data processing workflow is similar across multiple pl

In [26]:
d = DataLoader(f"_data\\testqa\\testdata_{1}.json")
r = next(d.get_data()) # record demo
print(r.data['Question'])
e = Evaluator()
e.record_lis = []
p1 = PredictorFactory().get_predictor(model_type='rag')
r.add_prediction(p1)
# r.data['Answer'] = r.prediction
e.add_Record(r)
e.evaluate()
print(r.evaluation['answer_correctness'])
print(r.evaluation['answer_similarity'])

What are the benefits and challenges of using synergistic sample preparation and multiple analytical techniques in untargeted metabolomics?


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

tp =  3
fp =  9
fn =  2
0.5068236708190633
0.9684711538644886


In [33]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
t = """
请你参考reference answer中的内容和格式,提高Original Answe的质量，最后只需要输出修改后的答案即可
***

Original Answer:{ori_a}
***
Reference Answer:{re_a}
"""
pr = PromptTemplate(
    input_variables=['ori_a','re_a','q'],
    template=t
)
llm = ChatOpenAI()
chain = pr |llm |StrOutputParser()

In [34]:
rewrite_a = chain.invoke({'ori_a':r.data['Answer'],'q':r.data['Question'],'re_a':r.prediction})

In [35]:
print(rewrite_a)

Based on the content provided, the benefits of using synergistic sample preparation and multiple analytical techniques in untargeted metabolomics include improved metabolite coverage, enhanced process for peak detection and metabolite identification, addressing reproducibility issues, enhancing data quality, and advancements in analytical techniques. However, challenges such as handling large data sets, software limitations, statistical analysis complexity, and uncertainty in metabolite identification need to be carefully addressed to ensure reliable and accurate results in metabolomics research. 

Improved Metabolite Coverage: Individual techniques can only characterize a few hundred metabolites, but a multiplatform approach using multiple analytical techniques enhances the number of reliably detected and correctly assigned metabolites. 

Enhanced Process: Synergistic sample preparation, along with the use of both non-destructive and destructive techniques, improves peak detection and

In [36]:
print(r.prediction)

Based on the provided document content, the benefits and challenges of using synergistic sample preparation and multiple analytical techniques in untargeted metabolomics can be summarized as follows:

**Benefits:**
1. **Improved Metabolite Coverage:** Individual techniques can only characterize a few hundred metabolites, but a multiplatform approach using multiple analytical techniques enhances the number of reliably detected and correctly assigned metabolites.
2. **Enhanced Process:** Synergistic sample preparation, along with the use of both non-destructive and destructive techniques, improves peak detection and metabolite identification strategies, leading to better annotation decisions.
3. **Addressing Reproducibility Issues:** Utilizing multiple analytical techniques helps in addressing reproducibility issues found in single platform methods.
4. **Data Quality Enhancement:** Liquid chromatography coupled with high resolution mass spectrometry enhances data quality in non-targeted 

In [37]:
e.record_lis[0].data['Answer'] = rewrite_a
e.evaluate()

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

tp =  2
fp =  0
fn =  9


In [38]:
print(r.evaluation['answer_correctness'])
print(r.evaluation['answer_similarity'])

0.478421734099564
0.9906100133213329
